In [ ]:
import os
import base64
import json
import pandas as pd

from ibm_watsonx_ai.foundation_models import ModelInference


In [ ]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",  # or your region
    "apikey": "Type your key here"
}

WATSONX_PROJECT_ID = "Enter Project ID here"


In [ ]:
# Vision model for image understanding
vision_model = ModelInference(
    model_id="meta-llama/llama-3-2-11b-vision-instruct",
    credentials=credentials,
    project_id=WATSONX_PROJECT_ID,
    params={
        "max_new_tokens": 256,
        "temperature": 0.0
    }
)

# Text-only Granite model for planning
planner_model = ModelInference(
    model_id="ibm/granite-3-2-8b-instruct",
    credentials=credentials,
    project_id=WATSONX_PROJECT_ID,
    params={
        "max_new_tokens": 1024,
        "temperature": 0.0
    }
)

print("Vision model:", vision_model.model_id)
print("Planner model:", planner_model.model_id)


In [ ]:
# 👇 Update these paths
image_filepaths = [
    "/Users/armenpischdotchian/Desktop/equip1.png",
    "/Users/armenpischdotchian/Desktop/equip2.png",
    "/Users/armenpischdotchian/Desktop/equip3.png",
]

images = []
filenames = []

for path in image_filepaths:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    with open(path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
    images.append(encoded)
    filenames.append(os.path.basename(path))

print(f"Loaded {len(images)} image(s):", filenames)


In [ ]:
def build_vision_messages(prompt: str, image_b64: str):
    data_url = f"data:image/png;base64,{image_b64}"
    return [
        {
            "role": "system",
            "content": (
                "You are a fitness assistant that analyzes images of gym equipment. "
                "You respond with a single line of text per image."
            ),
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": data_url},
                },
            ],
        },
    ]

vision_prompt = """
Look at the image of gym equipment.

Respond with EXACTLY ONE LINE in this format:
description | category | workout_type

Where:
- description: short text, e.g. "rack of black dumbbells on a stand"
- category: ONE of [barbell, dumbbell, machine, bodyweight, other]
- workout_type: ONE of [strength, endurance, flexibility, balance, cardio, other]

Rules:
- No extra text, no explanations, no bullets.
- Do NOT use the '|' character except as the separator.
"""

gym_equipment = []

for i, image in enumerate(images):
    messages = build_vision_messages(vision_prompt, image)
    response = vision_model.chat(messages=messages)
    line = response["choices"][0]["message"]["content"].strip()
    
    print(f"\nImage {i+1}: {filenames[i]}")
    print("Vision output:", line)

    parts = [p.strip() for p in line.split("|")]
    if len(parts) != 3:
        raise ValueError(f"Unexpected vision output format for {filenames[i]}: {line}")
    
    description, category, workout_type = parts

    gym_equipment.append({
        "filename": filenames[i],
        "description": description,
        "category": category,
        "workout_type": workout_type,
    })

print("\nDetected equipment:")
print(json.dumps(gym_equipment, indent=2))


In [ ]:
df_equipment = pd.DataFrame(gym_equipment)
df_equipment


In [ ]:
user_profile = {
    "name": "Alex",
    "age": 35,
    "gender": "unspecified",
    "fitness_level": "beginner",  # beginner / intermediate / advanced
    "experience_with_strength_training": "low",  # none / low / moderate / high
    "primary_goals": [
        "build strength",
        "improve general fitness"
    ],
    "days_per_week_available": 3,
    "session_length_minutes": 45,
    "injuries_or_limitations": [],
    "exercise_preferences": {
        "likes": ["free weights"],
        "dislikes": ["high impact jumping"]
    }
}

user_profile_json = json.dumps(user_profile, indent=2)
equipment_json = json.dumps(gym_equipment, indent=2)


In [ ]:
planner_prompt = f"""
You are an experienced, safety-conscious personal trainer.

You will create a 1-week workout plan using ONLY the equipment from the equipment list plus bodyweight.

USER PROFILE (JSON):
{user_profile_json}

AVAILABLE EQUIPMENT (JSON LIST):
{equipment_json}

Plan constraints:
- Use days_per_week_available from the user profile.
- Each day: warmup, 3–5 main exercises, cooldown.
- Exercises must match available equipment categories (or bodyweight).
- Adjust volume based on fitness_level and experience_with_strength_training.
- Respect injuries_or_limitations and exercise_preferences.

Output format:
You MUST respond as a plain text table with '|' separators.
First line is the header, then one line per exercise.

Columns (in this exact order):
Day | Focus | Exercise | Equipment | Sets | RepsOrTime | TargetMuscle | Intensity | Warmup | Cooldown | DayNotes

Rules:
- Do not include any explanations, markdown, or extra text.
- Do not add or remove columns.
- Every line (after the header) must have exactly 11 fields separated by '|'.
- Do not use '|' inside any field value.
"""

response = planner_model.chat(
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": planner_prompt}],
        }
    ]
)

plan_text = response["choices"][0]["message"]["content"]
print("Raw Granite planner output:\n")
print(plan_text)


In [ ]:
lines = [l.strip() for l in plan_text.splitlines() if l.strip()]

if not lines:
    raise ValueError("Planner output is empty.")

header_line = lines[0]
data_lines = lines[1:]

expected_cols = [
    "Day", "Focus", "Exercise", "Equipment", "Sets",
    "RepsOrTime", "TargetMuscle", "Intensity",
    "Warmup", "Cooldown", "DayNotes"
]

header_parts = [p.strip() for p in header_line.split("|")]
print("Header line parsed as:", header_parts)

# Optional: you can check if header matches expected_cols, but we won't be strict
rows = []

for line in data_lines:
    parts = [p.strip() for p in line.split("|")]
    if len(parts) != len(expected_cols):
        print("⚠️ Skipping malformed line:", line)
        continue
    row = dict(zip(expected_cols, parts))
    rows.append(row)

df_plan = pd.DataFrame(rows)
print("\n✅ Parsed workout plan into DataFrame:")
df_plan


In [ ]:
import pandas as pd
import os

# -----------------------------
# 1) Parse planner output (plan_text) into df_plan
# -----------------------------

if not isinstance(plan_text, str) or not plan_text.strip():
    raise ValueError("plan_text is empty or not a string. Make sure the planner_model cell ran correctly.")

# Split into non-empty lines
lines = [l.strip() for l in plan_text.splitlines() if l.strip()]

# Keep only lines that look like table rows (must contain at least one '|')
table_lines = [l for l in lines if "|" in l]

if not table_lines:
    raise ValueError("No table-like lines with '|' found in plan_text. Check the raw planner output.")

# First line is header, the rest are data
header_line = table_lines[0]
data_lines = table_lines[1:]

# Expected columns in order
expected_cols = [
    "Day", "Focus", "Exercise", "Equipment", "Sets",
    "RepsOrTime", "TargetMuscle", "Intensity",
    "Warmup", "Cooldown", "DayNotes"
]

header_parts = [p.strip() for p in header_line.split("|")]
print("Parsed header:", header_parts)

rows = []

for line in data_lines:
    parts = [p.strip() for p in line.split("|")]

    # If line has too many columns, merge the extras into the last column
    if len(parts) > len(expected_cols):
        parts = parts[:len(expected_cols)-1] + [" ".join(parts[len(expected_cols)-1:])]

    # If line has too few columns, pad with empty strings
    if len(parts) < len(expected_cols):
        parts = parts + [""] * (len(expected_cols) - len(parts))

    # If still mismatch (some weird row), skip
    if len(parts) != len(expected_cols):
        print("⚠️ Skipping malformed line:", line)
        continue

    row = dict(zip(expected_cols, parts))
    rows.append(row)

df_plan = pd.DataFrame(rows)

# Drop completely empty rows (just in case)
df_plan = df_plan.replace("", pd.NA).dropna(how="all").fillna("")

print("\n✅ Parsed workout plan into DataFrame:")
print(df_plan)
print("Shape:", df_plan.shape)

if df_plan.empty:
    raise ValueError("df_plan is empty after parsing. Check the planner output format.")


# -----------------------------
# 2) Build HTML with header block + per-day tables
# -----------------------------

# Normalize column names (strip extra spaces)
df_plan = df_plan.rename(columns=lambda c: c.strip() if isinstance(c, str) else c)

print("\nFinal df_plan columns:", list(df_plan.columns))

# Build header block from user_profile
goals_text = ", ".join(user_profile.get("primary_goals", []))

header_html = f"""
<h1>Personalized Workout Plan</h1>
<p><strong>Name:</strong> {user_profile.get("name", "N/A")}</p>
<p><strong>Age:</strong> {user_profile.get("age", "N/A")}</p>
<p><strong>Gender:</strong> {user_profile.get("gender", "N/A")}</p>
<p><strong>Fitness Level:</strong> {user_profile.get("fitness_level", "N/A")}</p>
<p><strong>Goals:</strong> {goals_text}</p>
<p><strong>Days/Week Available:</strong> {user_profile.get("days_per_week_available", "N/A")}</p>
<p><strong>Session Length:</strong> {user_profile.get("session_length_minutes", "N/A")} minutes</p>
"""

if user_profile.get("injuries_or_limitations"):
    header_html += (
        f"<p><strong>Injuries / Limitations:</strong> "
        f"{', '.join(user_profile['injuries_or_limitations'])}</p>"
    )

html_output = [header_html, "<hr>"]

# Columns we want to display in each exercise table
preferred_cols = [
    "Exercise", "Equipment", "Sets", "RepsOrTime", "TargetMuscle", "Intensity"
]

# Ensure there's a Day column (we created it from Granite's output, but just in case)
if "Day" not in df_plan.columns:
    df_plan["Day"] = "Plan"

for day_name in df_plan["Day"].unique():
    day_df = df_plan[df_plan["Day"] == day_name].copy()

    focus = day_df["Focus"].iloc[0] if "Focus" in day_df.columns else ""
    warmup = day_df["Warmup"].iloc[0] if "Warmup" in day_df.columns else ""
    cooldown = day_df["Cooldown"].iloc[0] if "Cooldown" in day_df.columns else ""
    notes = day_df["DayNotes"].iloc[0] if "DayNotes" in day_df.columns else ""

    # Only keep columns that actually exist
    display_cols = [c for c in preferred_cols if c in day_df.columns]

    html_output.append(f"<h2>{day_name}</h2>")
    if focus:
        html_output.append(f"<p><strong>Focus:</strong> {focus}</p>")
    if warmup:
        html_output.append(f"<p><strong>Warmup:</strong> {warmup}</p>")

    if display_cols:
        html_output.append(day_df[display_cols].to_html(index=False, border=1))
    else:
        # Fallback: show all columns
        html_output.append(day_df.to_html(index=False, border=1))

    if cooldown:
        html_output.append(f"<p><strong>Cooldown:</strong> {cooldown}</p>")
    if notes:
        html_output.append(f"<p><strong>Notes:</strong> {notes}</p><br>")

full_html = "\n".join(html_output)

output_path = "workout_plan.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"\n✅ Saved HTML to: {os.path.abspath(output_path)}")
